## Cyclic Iterators

In [ ]:
class CyclicIterator:
    def __init__(self, lst):
        self.lst = lst
        self.i = 0

    def __iter__(self):
        return self

    def __next__(self):
        result = self.lst[self.i % len(self.lst)]
        self.i += 1
        return result

In [ ]:
iter_cycl = CyclicIterator('NSWE')

for i in range(10):
    print(next(iter(iter_cycl)))

N
S
W
E
N
S
W
E
N
S


In [ ]:
import itertools

n = 10
iter_cycl = itertools.cycle('NSWE')
[f'{i}{next(iter_cycl)}' for i in range(1, n+1)]

['1N', '2S', '3W', '4E', '5N', '6S', '7W', '8E', '9N', '10S']

## Iterables with limited length

In [ ]:
class Squares:
  def __init__(self, length):
    self._length = length

  def __iter__(self):
    return self.SquareIterator(self._length)

  class SquareIterator:
    def __init__(self, length):
      self._length = length
      self.i = 0

    def __iter__(self):
      return self

    def __next__(self):
      if self.i >= self._length:
        raise StopIteration
      else:
        result = self.i ** 2
        self.i += 1
        return result

In [ ]:
square_iterables = Squares(5)

for num in square_iterables:
  print(num)

0
1
4
9
16


## Lazy Iterables

In [ ]:
class Squares:
  def __iter__(self):
    return self.SquareIterator()

  class SquareIterator:
    def __init__(self):
      self.i = 0

    def __iter__(self):
      return self

    def __next__(self):
      result = self.i ** 2
      self.i += 1
      return result

In [ ]:
square_lazy_iterables = Squares()
square_lazy_iterator = iter(square_lazy_iterables)

for _ in range(10):
    print(next(square_lazy_iterator))

0
1
4
9
16
25
36
49
64
81


---

## In-built Iterables

In [ ]:
def is_iterable(obj: 'Python Object') -> bool:
    """"
    Function to check if an obejct is Iterable or not
    :param obj: object of a class (In Python since everything is an object, integers, floats, lists, etc can be passed as objects as well)
    :return: Boolean value (True/False)
    """
    try:
        iter(obj)
        return True
    except TypeError:
        return False

In [ ]:
print(is_iterable(range(1)))
print(is_iterable(enumerate(range(10))))
print(is_iterable(zip(range(10), range(11, 21))))

True
True
True


### `range()`
- It is a in-built **lazy iterable**
- We can initialize large number of values but computation will only be done when it is called
- The execution proves that it is lazy iterable

In [ ]:
%%timeit
l = range(10_000_000)
l

The slowest run took 5.72 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 5: 245 ns per loop


In [ ]:
%%timeit
l = list(range(10_000_000))
l

1 loop, best of 5: 285 ms per loop


### `zip()`
- This is another lazy iterable available in Python
- It will create an object which can zip the data of multiple lists but only executes them when it is called on the zip object
- The execution time difference shows the lazy nature of zip iterable

In [ ]:
l1 = range(10)
l2 = range(11, 21)

z = zip(l1, l2)

In [ ]:
%%timeit
z

The slowest run took 34.28 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 5: 34.2 ns per loop


In [ ]:
%%timeit
list(z)

The slowest run took 16.91 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 5: 220 ns per loop


### `enumerate()`
- This is another lazy iterable available in Python
- It will create an object which can enumerate the data but only executes them when it is called on the enumerate object
- The execution time difference shows the lazy nature of enumerate iterable

In [ ]:
l1 = list(range(100))
l1.reverse()

en = enumerate(l1)

In [ ]:
%%timeit
en

The slowest run took 36.63 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 5: 35.5 ns per loop


In [ ]:
%%timeit
list(en)

The slowest run took 57.82 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 5: 217 ns per loop


---

## Sorting Iterables

In [ ]:
import random

class RandomInts:
    def __init__(self, length, *, seed=0, lower=0, upper=10):
        self.length = length
        self.seed = seed
        self.lower = lower
        self.upper = upper
        
    def __len__(self):
        return self.length
    
    def __iter__(self):
        return self.RandomIterator(self.length, 
                                   seed = self.seed, 
                                   lower = self.lower,
                                   upper=self.upper)
    
    
    class RandomIterator:
        def __init__(self, length, *, seed, lower, upper):
            self.length = length
            self.lower = lower
            self.upper = upper
            self.num_requests = 0
            random.seed(seed)
            
        def __iter__(self):
            return self
        
        def __next__(self):
            if self.num_requests >= self.length:
                raise StopIteration
            else:
                result = random.randint(self.lower, self.upper)
                self.num_requests += 1
                return result

In [ ]:
randoms = RandomInts(10)

In [ ]:
sorted(randoms)

[0, 4, 4, 5, 6, 6, 6, 7, 7, 8]

In [ ]:
sorted(randoms, reverse=True)

[8, 7, 7, 6, 6, 6, 5, 4, 4, 0]

---

## Iterating Callable




In [ ]:
def counter():
    i = 0
    def inc():
        nonlocal i 
        i += 1
        return i
    return inc

In [ ]:
class CounterIterator:
    def __init__(self, counter_callable, sentinel):
        self.counter_callable = counter_callable
        self.sentinel = sentinel
        self.is_consumed = False

    def __iter__(self):
        return self

    def __next__(self):
        if self.is_consumed:
            raise StopIteration
        else:
            result = self.counter_callable()
            if result == self.sentinel:
                self.is_consumed = True
                raise StopIteration

            else:
                return result


In [ ]:
cnt = counter()
cnt_iter = CounterIterator(cnt, 5)

for c in cnt_iter:
    print(c)

1
2
3
4


---

## Delegating Iterators

In [ ]:
from collections import namedtuple

Person = namedtuple('Person', 'first last')

In [ ]:
class PersonNames:
    def __init__(self, persons):
        try:
            self._persons = [person.first.capitalize()
                             + ' ' + person.last.capitalize()
                            for person in persons]
        except (TypeError, AttributeError):
            self._persons = []

    def __iter__(self):
        return iter(self._persons)

In [ ]:
persons = [Person('michaeL', 'horDON'), Person('srikanth', 'latify'), Person('maX', 'VertaPPEn')]

person_names = PersonNames(persons)

In [ ]:
for p in person_names:
    print(p)

Michael Hordon
Srikanth Latify
Max Vertappen


In [ ]:
[tuple(person_name.split()) for person_name in sorted(person_names)]

[('Max', 'Vertappen'), ('Michael', 'Hordon'), ('Srikanth', 'Latify')]

---

## Reversed Iteration

In [ ]:
_SUITS = ('Spades', 'Hearts', 'Diamonds', 'Clubs')
_RANKS = tuple(range(2, 11) ) + ('J', 'Q', 'K', 'A')
from collections import namedtuple

Card = namedtuple('Card', 'rank suit')

class CardDeck:
    def __init__(self):
        self.length = len(_SUITS) * len(_RANKS)

    def __len__(self):
        return self.length
    
    def __iter__(self):
        return self.CardDeckIterator(self.length)
        
    def __reversed__(self):
        return self.CardDeckIterator(self.length, reverse=True)
    
    class CardDeckIterator:
        def __init__(self, length, *, reverse=False):
            self.length = length
            self.reverse = reverse
            self.i = 0
            
        def __iter__(self):
            return self
        
        def __next__(self):
            if self.i >= self.length:
                raise StopIteration
            else:
                if self.reverse:
                    index = self.length -1 - self.i
                else:
                    index = self.i
                suit = _SUITS[index // len(_RANKS)]
                rank = _RANKS[index % len(_RANKS)]
                self.i += 1
                return Card(rank, suit)

In [ ]:
deck = CardDeck()

In [ ]:
for card in deck:
    print(card)

Card(rank=2, suit='Spades')
Card(rank=3, suit='Spades')
Card(rank=4, suit='Spades')
Card(rank=5, suit='Spades')
Card(rank=6, suit='Spades')
Card(rank=7, suit='Spades')
Card(rank=8, suit='Spades')
Card(rank=9, suit='Spades')
Card(rank=10, suit='Spades')
Card(rank='J', suit='Spades')
Card(rank='Q', suit='Spades')
Card(rank='K', suit='Spades')
Card(rank='A', suit='Spades')
Card(rank=2, suit='Hearts')
Card(rank=3, suit='Hearts')
Card(rank=4, suit='Hearts')
Card(rank=5, suit='Hearts')
Card(rank=6, suit='Hearts')
Card(rank=7, suit='Hearts')
Card(rank=8, suit='Hearts')
Card(rank=9, suit='Hearts')
Card(rank=10, suit='Hearts')
Card(rank='J', suit='Hearts')
Card(rank='Q', suit='Hearts')
Card(rank='K', suit='Hearts')
Card(rank='A', suit='Hearts')
Card(rank=2, suit='Diamonds')
Card(rank=3, suit='Diamonds')
Card(rank=4, suit='Diamonds')
Card(rank=5, suit='Diamonds')
Card(rank=6, suit='Diamonds')
Card(rank=7, suit='Diamonds')
Card(rank=8, suit='Diamonds')
Card(rank=9, suit='Diamonds')
Card(rank=10, 

In [ ]:
deck = reversed(CardDeck())
for card in deck:
    print(card)

Card(rank='A', suit='Clubs')
Card(rank='K', suit='Clubs')
Card(rank='Q', suit='Clubs')
Card(rank='J', suit='Clubs')
Card(rank=10, suit='Clubs')
Card(rank=9, suit='Clubs')
Card(rank=8, suit='Clubs')
Card(rank=7, suit='Clubs')
Card(rank=6, suit='Clubs')
Card(rank=5, suit='Clubs')
Card(rank=4, suit='Clubs')
Card(rank=3, suit='Clubs')
Card(rank=2, suit='Clubs')
Card(rank='A', suit='Diamonds')
Card(rank='K', suit='Diamonds')
Card(rank='Q', suit='Diamonds')
Card(rank='J', suit='Diamonds')
Card(rank=10, suit='Diamonds')
Card(rank=9, suit='Diamonds')
Card(rank=8, suit='Diamonds')
Card(rank=7, suit='Diamonds')
Card(rank=6, suit='Diamonds')
Card(rank=5, suit='Diamonds')
Card(rank=4, suit='Diamonds')
Card(rank=3, suit='Diamonds')
Card(rank=2, suit='Diamonds')
Card(rank='A', suit='Hearts')
Card(rank='K', suit='Hearts')
Card(rank='Q', suit='Hearts')
Card(rank='J', suit='Hearts')
Card(rank=10, suit='Hearts')
Card(rank=9, suit='Hearts')
Card(rank=8, suit='Hearts')
Card(rank=7, suit='Hearts')
Card(ran

In [ ]:
class Squares:
    def __init__(self, length):
        self.length = length
        self.squares = [i **2 for i in range(length)]
        
    def __len__(self):
        return len(self.squares)
    
    def __getitem__(self, s):
        return self.squares[s]
    
    def __reversed__(self):
        print('__reversed__ called')
        return [i for i in range(self.length-1, -1, -1)]

---

## Iterators as function arguments

In [ ]:
import random

class Randoms:
    def __init__(self, n):
        self.n = n
        self.i = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.i >= self.n:
            raise StopIteration
        else:
            self.i += 1
            return random.randint(0, 100)

In [ ]:
random.seed(0)
l = Randoms(10)

min(l)

5

In [ ]:
max(l)

ValueError: ignored